In [1]:
"""
  ####### Description #######
  This jupyter notebook was an attempt to run the ACMA 2016 authored by Jun Xiong, et al.
  Jupyter notebook author: Mary Grace Barbacias
  All resources were sourced from the course materials and the open-source websites (FAO, USGS, GEE)
"""

# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('Installing geemap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

The above code installs geemap into the environment using pip install. The chunk below is just importing the required libraries. Make sure to change the username and project_id to your ee username and project_id.

In [2]:
import ee
import geemap
import jupyter_leaflet

ee.Authenticate()

project_id = 'geospatial-python-459812' #change this to your project ID
username = 'gracebarbacias'  # change this to your username
# Initialize Earth Engine with the project ID
ee.Initialize(project=project_id)


###Preprocessing Raster files###
##
This part is for preprocessing the raster files downloaded from GAEZ Portal. Link: https://gaez.fao.org/pages/data-viewer
Specifications: 
    Sub-Theme Name isL Growing Period
    Variable name is: Total number of growing period days
    Time Period is: 1971-2000 
    Climate Data Source is: CRUTS32
    RCP is: Historical

The above specification is for the baseline used for 2014 version of ACMA which used GAEZ v.3. 

To avoid replicating the same process of downloading these files, I hosted them on my github, accessible here: 
baseline TIF for 1971-2000: https://github.com/GraceBarbacias/sources/blob/main/lgd_CRUTS32_Hist_7100.tif

After downloading, we check how it looks like.


In [5]:
import requests
import os
import geemap
import rasterio
import numpy as np

# Download TIFs from GitHub if not already present
sources_dir = os.path.join(os.getcwd(), "sources")
os.makedirs(sources_dir, exist_ok=True)

AEZ_2000_url = "https://github.com/GraceBarbacias/sources/raw/main/lgd_CRUTS32_Hist_7100.tif"
AEZ_2000 = os.path.join(sources_dir, "lgd_CRUTS32_Hist_7100.tif")

def download_if_not_exists(url, out_path):
    if not os.path.exists(out_path):
        print(f"Downloading {os.path.basename(out_path)} ...")
        r = requests.get(url, stream=True)
        with open(out_path, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
        print(f"Saved to {out_path}")

download_if_not_exists(AEZ_2000_url, AEZ_2000)

# List of TIFs to explore
raster_files = [
    ("AEZ 2000", AEZ_2000)
]

for label, tif_path in raster_files:
    print(f"{label} info:")
    print(tif_path)
    with rasterio.open(tif_path) as src:
        print("CRS:", src.crs)
        print("Bounds:", src.bounds)
        print("Width, Height:", src.width, src.height)
        print("Count (bands):", src.count)
        print("Data type:", src.dtypes)
        print("---")
        src_array = src.read(1)
        print("Min value:", np.min(src_array), "Max value:", np.max(src_array), "Mean value:", np.mean(src_array))

        # Print unique values in the raster (like an attribute table for categorical rasters)
        unique, counts = np.unique(src_array, return_counts=True)
        

        print("Value\tCount")
        for val, cnt in list(zip(unique, counts))[:10]:  # cap at 10 lines
            print(f"{val}\t{cnt}")
        if len(unique) > 10:
            print(f"... ({len(unique)-10} more values)")


AEZ 2000 info:
/Users/au783075/Documents/GitHub/sources/sources/lgd_CRUTS32_Hist_7100.tif
CRS: EPSG:4326
Bounds: BoundingBox(left=-180.0, bottom=-90.0, right=180.0, top=90.0)
Width, Height: 4320 2160
Count (bands): 1
Data type: ('int16',)
---
Min value: -9 Max value: 366 Mean value: 26.76743066272291
Value	Count
-9	7053626
0	333035
1	6093
2	5395
3	5131
4	4826
5	4421
6	3887
7	4250
8	3937
... (358 more values)


###Defining zone boundaries###

According to [Xiong, et al., (2017)](https://doi.org/10.1016/j.isprsjprs.2017.01.019), the 15 GAEZ were unified into 8, with the Length of Growing Period ranges detailed in Fig.1 of the paper. Hence we also do that by grouping the values into the 8 ranges they specified, proceed to cropping into the African region. Values are as follows: -9 (seas, ocean), 1 to 8 are different FAO AEZ zones defined by length of growing period.

In [6]:
from rasterio.windows import from_bounds
import rasterio as rio

#only run this ONCE, since it will reclassify the zones inaccurately if run multiple times
for label, tif_path in raster_files:
    with rasterio.open(tif_path) as src:
        # Read the raster data first
        src_array = src.read(1)
        
        # Define zone conditions based on growing period days
        zone_conditions = [
            (src_array == 0),                              # Zone 1: 0 days
            (src_array >= 1) & (src_array <= 59),         # Zone 2: 1-59 days
            (src_array >= 60) & (src_array <= 119),       # Zone 3: 60-119 days
            (src_array >= 120) & (src_array <= 179),      # Zone 4: 120-179 days
            (src_array >= 180) & (src_array <= 239),      # Zone 5: 180-239 days
            (src_array >= 240) & (src_array <= 299),      # Zone 6: 240-299 days
            (src_array >= 300) & (src_array <= 364),      # Zone 7: 300-364 days
            (src_array >= 365)                            # Zone 8: 365+ days
        ]
        zone_labels = [1, 2, 3, 4, 5, 6, 7, 8]

        # Assign zones, keep -9 as nodata
        zones = np.select(zone_conditions, zone_labels, default=-9)

        # Print a summary of the classified zones
        unique_zones, zone_counts = np.unique(zones, return_counts=True)
        print(f"\n{label} - Zone classification summary:")
        print("Zone\tCount")
        for z, c in zip(unique_zones, zone_counts):
            print(f"{z}\t{c}")

        # Cropping the zones to Africa
        # Define African region bounds (approximate)
        lat_min, lat_max = -35, 37
        lon_min, lon_max = -20, 52

        # Calculate the window for these bounds using the raster's transform
        window = from_bounds(
            left=lon_min, bottom=lat_min, right=lon_max, top=lat_max,
            transform=src.transform
        )

        # Crop the zones array
        zones_africa = zones[
            int(window.row_off):int(window.row_off + window.height),
            int(window.col_off):int(window.col_off + window.width)
        ]

        # Create dynamic output filename based on input
        filename = os.path.basename(tif_path).replace('.tif', '_zones_africa.tif')
        output_path = os.path.join('maps', 'FAO_GAEZ', filename)
        
        # Create output directory if it doesn't exist
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        # Save the unified and cropped zones as a new GeoTIFF
        with rasterio.open(
            output_path,
            'w',
            driver='GTiff',
            height=zones_africa.shape[0],
            width=zones_africa.shape[1],
            count=1,
            dtype=zones_africa.dtype,
            crs=src.crs,
            transform=rasterio.windows.transform(window, src.transform),
            nodata=-9
        ) as dst:
            dst.write(zones_africa, 1)
            
        print(f"Saved zones for {label} to: {output_path}")



AEZ 2000 - Zone classification summary:
Zone	Count
-9	7053626
1	333035
2	284363
3	519337
4	468337
5	239251
6	167568
7	112882
8	152801
Saved zones for AEZ 2000 to: maps/FAO_GAEZ/lgd_CRUTS32_Hist_7100_zones_africa.tif


We then save these into geojson and then upload in ee as FeatureCollection. I know that this is a long method of doing things, but I tried directly uploading the image asset and do the convertion to vector in ee and it fails due to the size. 

In [7]:
from rasterio.features import shapes
import geopandas as gpd
import os
import rasterio as rio

#new paths
# Define file paths
output_dir = os.path.join(os.getcwd(), "maps", "FAO_GAEZ")
AEZ_2000_zones = os.path.join(output_dir, "lgd_CRUTS32_Hist_7100_zones_africa.tif")

# List of TIFs to explore
raster_files_zones = [
    ("AEZ 2000 zones", AEZ_2000_zones)
]

for label, tif_path in raster_files_zones:
    with rio.open(tif_path) as src:
        zones_africa = src.read(1)
        # Mask: True where not nodata (-9), False where nodata
        mask = zones_africa != -9
        transform = src.transform
        crs = src.crs

        # Convert to supported dtype
        zones_africa = zones_africa.astype('int16')

        # Vectorize: extract polygons for each zone
        results = (
            {'properties': {'region': int(v)}, 'geometry': s}
            for s, v in shapes(zones_africa, mask=mask, transform=transform)
        )

        # Convert to GeoDataFrame
        gdf = gpd.GeoDataFrame.from_features(list(results), crs=crs)

    # Create dynamic output filename based on input
    filename = os.path.basename(tif_path).replace('_zones_africa.tif', '.geojson')
    geojson_output_dir = os.path.join(os.getcwd(), "maps", "geojson")
    
    # Create output directory if it doesn't exist
    os.makedirs(geojson_output_dir, exist_ok=True)
    
    # Save as GeoJSON (EE-friendly upload format)
    geojson_path = os.path.join(geojson_output_dir, filename)
    gdf.to_file(geojson_path, driver='GeoJSON')
    
    print(f"Saved GeoJSON for {label} to: {geojson_path}")

Saved GeoJSON for AEZ 2000 zones to: /Users/au783075/Documents/GitHub/sources/maps/geojson/lgd_CRUTS32_Hist_7100.geojson


We need to convert the geojson into feature collection and upload as FeatureCollection in ee. The next code chunk does the conversion. 
Please replace the assetId. We do this for the 2000 geojson file. 
This process takes some time to finish. You have to monitor the progress in Tasks tab in your ee page.


In [8]:
import json
import os
from pathlib import Path

# Define file paths
output_dir = os.path.join(os.getcwd(), "maps", "geojson")
geojson_files_zones = [
    ("AEZ_2000_zones", os.path.join(output_dir, "lgd_CRUTS32_Hist_7100.geojson"))
]

# Process each GeoJSON file
for label, geojson_path in geojson_files_zones:
    print(f"\nProcessing {label}...")
    
    if not Path(geojson_path).exists():
        print(f"Warning: File not found - {geojson_path}")
        continue
    
    try:
        with open(geojson_path, 'r') as f:
            geojson_dict = json.load(f)
        
        print(f"✓ Loaded GeoJSON with {len(geojson_dict.get('features', []))} features")
        
        ee_object = geemap.geojson_to_ee(geojson_dict)
        asset_id = f"users/{username}/{label}"
        
        export_task = ee.batch.Export.table.toAsset(
            collection=ee_object,
            description=label,
            assetId=asset_id
        )
        
        export_task.start()
        print(f"✓ Started export task: {asset_id}")
        
    except Exception as e:
        print(f"Error processing {label}: {e}")

print("\n=== Check Earth Engine Tasks tab to monitor progress ===")



Processing AEZ_2000_zones...
✓ Loaded GeoJSON with 1022 features
✓ Started export task: users/gracebarbacias/AEZ_2000_zones

=== Check Earth Engine Tasks tab to monitor progress ===


About the next part, it was mentioned that the input MODIS images' non-cropping areas were masked out using the cropland layer, and then clustered using the FAO zones we previously processed.

Since I have no access to the actual RCL2014 250m layer that they used, we will use the 1km resolution global cropland layer for 2010 called [GFSAD1KCM](https://lpdaac.usgs.gov/products/gfsad1kcmv001/.) and resample it to 250m and project to the same CRS.


In [9]:
from IPython.display import display, Markdown


# Print band information and metadata for the cropland mask
info = ee.Image('USGS/GFSAD1000_V1').getInfo()
bands = info['bands']
print("Band Information for USGS/GFSAD1000_V1:")
for band in bands:
    print(f"Band: {band['id']}")
    if 'description' in band:
        print(f"  Description: {band['description']}")
    if 'properties' in band:
        for k, v in band['properties'].items():
            print(f"  {k}: {v}")
    print()

display(Markdown("""
| Value | Class Name                                  |
|-------|---------------------------------------------|
| 0     | Non croplands                               |
| 1     | Croplands: irrigation major                 |
| 2     | Croplands: irrigation minor                 |
| 3     | Croplands: rainfed                          |
| 4     | Croplands: rainfed, minor fragments         |
| 5     | Croplands: rainfed, very minor fragments    |
"""))

# Map the cropland mask
#load irrigated vs non irrigated map
cropland_mask = ee.Image('USGS/GFSAD1000_V1')

Map = geemap.Map(center=[0, 20], zoom=3)
Map.addLayer(
    cropland_mask,
    {'min': 0, 'max': 5, 'palette': ['black', 'orange', 'brown', '02a50f', 'green', 'yellow']},
    'GFSAD1000 v1 Cropland Mask'
)
Map


Band Information for USGS/GFSAD1000_V1:
Band: landcover




| Value | Class Name                                  |
|-------|---------------------------------------------|
| 0     | Non croplands                               |
| 1     | Croplands: irrigation major                 |
| 2     | Croplands: irrigation minor                 |
| 3     | Croplands: rainfed                          |
| 4     | Croplands: rainfed, minor fragments         |
| 5     | Croplands: rainfed, very minor fragments    |


Map(center=[0, 20], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…

##Running the Automated Cropland Mapping Algorithm (ACMA) for 2014##

All the above steps were merely preparations for the following code chunks. We needed the zone boundaries for 2014, and that's the AEZ_2000. 

The code chunk below uses the cropland layer (raster) as base and the zones (FeatureCollection) as source for 'region' attribute value. Also contains the decision trees and the ACMA code. We implement it in the next code chunk.

In [10]:
center = ee.Geometry.Point([2.3212, 36.2099])

# Define Africa bounds and geometry
lat_min, lat_max = -35, 37
lon_min, lon_max = -20, 52
africa_geom = ee.Geometry.Rectangle([lon_min, lat_min, lon_max, lat_max])

# Resample cropland mask to 250m and clip to Africa
cropland_mask_250m = cropland_mask \
    .resample('bilinear') \
    .reproject(crs='EPSG:4326', scale=250) \
    .clip(africa_geom)


def classify_cropland_by_zone(cropland_mask, zones_fc, valid_values):
    # Step 1: Mask the cropland raster to only include valid values
    cropland_masked = cropland_mask.remap(valid_values, [1]*len(valid_values)).selfMask()
    
    # Step 2: Rasterize the zones (assign zone 'region' values to pixels)
    zones_raster = zones_fc.reduceToImage(['region'], ee.Reducer.first())
    
    # Step 3: Mask the zones raster by the cropland pixels
    zones_cropland = zones_raster.updateMask(cropland_masked)
    
    # Step 4: Convert the result to vectors (regions will be grouped by zone)
    vectors = zones_cropland.reduceToVectors(
        geometry = zones_fc.geometry(),
        scale = 250,
        geometryType = 'polygon',
        eightConnected = False,
        labelProperty = 'region',
        maxPixels = 1e13
    )
    return vectors

# Define decision tree strings for each zone
treeString = ee.List([
    None,  # Index 0 - null
    # Zone 1
    """1) root 5111 3300 0 (0.35 0.0029 0.098 0.19 0.13 0.21 0.0033 0.014)  
  2) B4>=60.5 1750  229 0 (0.87 0.00057 0.041 0.0011 0.043 0.045 0 0.00057) *
  3) B4< 60.5 3361 2349 5 (0.086 0.0042 0.13 0.28 0.17 0.3 0.0051 0.021)  
    6) B4>=17.5 2607 1701 5 (0.11 0.0054 0.16 0.15 0.21 0.35 0.0065 0.013)  
      12) B4>=29.5 1325  858 5 (0.2 0.0091 0.22 0.047 0.16 0.35 0.011 0.0083)  
        24) B5>=76.5 419  248 2 (0.095 0.0024 0.41 0.036 0.14 0.31 0.0072 0.0024) *
        25) B5< 76.5 906  567 5 (0.24 0.012 0.13 0.052 0.16 0.37 0.012 0.011)  
          50) B3>=37.5 205   85 0 (0.59 0.015 0.059 0.02 0.11 0.18 0.02 0.015) *
          51) B3< 37.5 701  399 5 (0.14 0.011 0.16 0.061 0.18 0.43 0.01 0.01) *
      13) B4< 29.5 1282  843 3 (0.023 0.0016 0.094 0.26 0.26 0.34 0.0023 0.018) *
    7) B4< 17.5 754  202 3 (0.0013 0 0.017 0.73 0.06 0.14 0 0.049) *""",
    # Zone 2
    """1) root 5111 3300 0 (0.35 0.0029 0.098 0.19 0.13 0.21 0.0033 0.014)  
  2) B4>=60.5 1750  229 0 (0.87 0.00057 0.041 0.0011 0.043 0.045 0 0.00057) *
  3) B4< 60.5 3361 2349 5 (0.086 0.0042 0.13 0.28 0.17 0.3 0.0051 0.021)  
    6) B4>=17.5 2607 1701 5 (0.11 0.0054 0.16 0.15 0.21 0.35 0.0065 0.013)  
      12) B4>=29.5 1325  858 5 (0.2 0.0091 0.22 0.047 0.16 0.35 0.011 0.0083)  
        24) B5>=76.5 419  248 2 (0.095 0.0024 0.41 0.036 0.14 0.31 0.0072 0.0024) *
        25) B5< 76.5 906  567 5 (0.24 0.012 0.13 0.052 0.16 0.37 0.012 0.011)  
          50) B3>=37.5 205   85 0 (0.59 0.015 0.059 0.02 0.11 0.18 0.02 0.015) *
          51) B3< 37.5 701  399 5 (0.14 0.011 0.16 0.061 0.18 0.43 0.01 0.01) *
      13) B4< 29.5 1282  843 3 (0.023 0.0016 0.094 0.26 0.26 0.34 0.0023 0.018) *
    7) B4< 17.5 754  202 3 (0.0013 0 0.017 0.73 0.06 0.14 0 0.049) *""",
    # Zone 3
    """1) root 5111 3300 0 (0.35 0.0029 0.098 0.19 0.13 0.21 0.0033 0.014)  
  2) B4>=60.5 1750  229 0 (0.87 0.00057 0.041 0.0011 0.043 0.045 0 0.00057) *
  3) B4< 60.5 3361 2349 5 (0.086 0.0042 0.13 0.28 0.17 0.3 0.0051 0.021)  
    6) B4>=17.5 2607 1701 5 (0.11 0.0054 0.16 0.15 0.21 0.35 0.0065 0.013)  
      12) B4>=29.5 1325  858 5 (0.2 0.0091 0.22 0.047 0.16 0.35 0.011 0.0083)  
        24) B5>=76.5 419  248 2 (0.095 0.0024 0.41 0.036 0.14 0.31 0.0072 0.0024) *
        25) B5< 76.5 906  567 5 (0.24 0.012 0.13 0.052 0.16 0.37 0.012 0.011)  
          50) B3>=37.5 205   85 0 (0.59 0.015 0.059 0.02 0.11 0.18 0.02 0.015) *
          51) B3< 37.5 701  399 5 (0.14 0.011 0.16 0.061 0.18 0.43 0.01 0.01) *
      13) B4< 29.5 1282  843 3 (0.023 0.0016 0.094 0.26 0.26 0.34 0.0023 0.018) *
    7) B4< 17.5 754  202 3 (0.0013 0 0.017 0.73 0.06 0.14 0 0.049) *""",
    # Zone 4
    """1) root 5111 3300 0 (0.35 0.0029 0.098 0.19 0.13 0.21 0.0033 0.014)  
  2) B4>=60.5 1750  229 0 (0.87 0.00057 0.041 0.0011 0.043 0.045 0 0.00057) *
  3) B4< 60.5 3361 2349 5 (0.086 0.0042 0.13 0.28 0.17 0.3 0.0051 0.021)  
    6) B4>=17.5 2607 1701 5 (0.11 0.0054 0.16 0.15 0.21 0.35 0.0065 0.013)  
      12) B4>=29.5 1325  858 5 (0.2 0.0091 0.22 0.047 0.16 0.35 0.011 0.0083)  
        24) B5>=76.5 419  248 2 (0.095 0.0024 0.41 0.036 0.14 0.31 0.0072 0.0024) *
        25) B5< 76.5 906  567 5 (0.24 0.012 0.13 0.052 0.16 0.37 0.012 0.011)  
          50) B3>=37.5 205   85 0 (0.59 0.015 0.059 0.02 0.11 0.18 0.02 0.015) *
          51) B3< 37.5 701  399 5 (0.14 0.011 0.16 0.061 0.18 0.43 0.01 0.01) *
      13) B4< 29.5 1282  843 3 (0.023 0.0016 0.094 0.26 0.26 0.34 0.0023 0.018) *
    7) B4< 17.5 754  202 3 (0.0013 0 0.017 0.73 0.06 0.14 0 0.049) *""",
    # Zone 5
    """1) root 5111 3300 0 (0.35 0.0029 0.098 0.19 0.13 0.21 0.0033 0.014)  
  2) B4>=60.5 1750  229 0 (0.87 0.00057 0.041 0.0011 0.043 0.045 0 0.00057) *
  3) B4< 60.5 3361 2349 5 (0.086 0.0042 0.13 0.28 0.17 0.3 0.0051 0.021)  
    6) B4>=17.5 2607 1701 5 (0.11 0.0054 0.16 0.15 0.21 0.35 0.0065 0.013)  
      12) B4>=29.5 1325  858 5 (0.2 0.0091 0.22 0.047 0.16 0.35 0.011 0.0083)  
        24) B5>=76.5 419  248 2 (0.095 0.0024 0.41 0.036 0.14 0.31 0.0072 0.0024) *
        25) B5< 76.5 906  567 5 (0.24 0.012 0.13 0.052 0.16 0.37 0.012 0.011)  
          50) B3>=37.5 205   85 0 (0.59 0.015 0.059 0.02 0.11 0.18 0.02 0.015) *
          51) B3< 37.5 701  399 5 (0.14 0.011 0.16 0.061 0.18 0.43 0.01 0.01) *
      13) B4< 29.5 1282  843 3 (0.023 0.0016 0.094 0.26 0.26 0.34 0.0023 0.018) *
    7) B4< 17.5 754  202 3 (0.0013 0 0.017 0.73 0.06 0.14 0 0.049) *""",
    # Zone 6
    """1) root 5111 3300 0 (0.35 0.0029 0.098 0.19 0.13 0.21 0.0033 0.014)  
  2) B4>=60.5 1750  229 0 (0.87 0.00057 0.041 0.0011 0.043 0.045 0 0.00057) *
  3) B4< 60.5 3361 2349 5 (0.086 0.0042 0.13 0.28 0.17 0.3 0.0051 0.021)  
    6) B4>=17.5 2607 1701 5 (0.11 0.0054 0.16 0.15 0.21 0.35 0.0065 0.013)  
      12) B4>=29.5 1325  858 5 (0.2 0.0091 0.22 0.047 0.16 0.35 0.011 0.0083)  
        24) B5>=76.5 419  248 2 (0.095 0.0024 0.41 0.036 0.14 0.31 0.0072 0.0024) *
        25) B5< 76.5 906  567 5 (0.24 0.012 0.13 0.052 0.16 0.37 0.012 0.011)  
          50) B3>=37.5 205   85 0 (0.59 0.015 0.059 0.02 0.11 0.18 0.02 0.015) *
          51) B3< 37.5 701  399 5 (0.14 0.011 0.16 0.061 0.18 0.43 0.01 0.01) *
      13) B4< 29.5 1282  843 3 (0.023 0.0016 0.094 0.26 0.26 0.34 0.0023 0.018) *
    7) B4< 17.5 754  202 3 (0.0013 0 0.017 0.73 0.06 0.14 0 0.049) *""",
    # Zone 7
    """1) root 5111 3300 0 (0.35 0.0029 0.098 0.19 0.13 0.21 0.0033 0.014)  
  2) B4>=60.5 1750  229 0 (0.87 0.00057 0.041 0.0011 0.043 0.045 0 0.00057) *
  3) B4< 60.5 3361 2349 5 (0.086 0.0042 0.13 0.28 0.17 0.3 0.0051 0.021)  
    6) B4>=17.5 2607 1701 5 (0.11 0.0054 0.16 0.15 0.21 0.35 0.0065 0.013)  
      12) B4>=29.5 1325  858 5 (0.2 0.0091 0.22 0.047 0.16 0.35 0.011 0.0083)  
        24) B5>=76.5 419  248 2 (0.095 0.0024 0.41 0.036 0.14 0.31 0.0072 0.0024) *
        25) B5< 76.5 906  567 5 (0.24 0.012 0.13 0.052 0.16 0.37 0.012 0.011)  
          50) B3>=37.5 205   85 0 (0.59 0.015 0.059 0.02 0.11 0.18 0.02 0.015) *
          51) B3< 37.5 701  399 5 (0.14 0.011 0.16 0.061 0.18 0.43 0.01 0.01) *
      13) B4< 29.5 1282  843 3 (0.023 0.0016 0.094 0.26 0.26 0.34 0.0023 0.018) *
    7) B4< 17.5 754  202 3 (0.0013 0 0.017 0.73 0.06 0.14 0 0.049) *""",
    # Zone 8
    """1) root 5111 3300 0 (0.35 0.0029 0.098 0.19 0.13 0.21 0.0033 0.014)  
  2) B4>=60.5 1750  229 0 (0.87 0.00057 0.041 0.0011 0.043 0.045 0 0.00057) *
  3) B4< 60.5 3361 2349 5 (0.086 0.0042 0.13 0.28 0.17 0.3 0.0051 0.021)  
    6) B4>=17.5 2607 1701 5 (0.11 0.0054 0.16 0.15 0.21 0.35 0.0065 0.013)  
      12) B4>=29.5 1325  858 5 (0.2 0.0091 0.22 0.047 0.16 0.35 0.011 0.0083)  
        24) B5>=76.5 419  248 2 (0.095 0.0024 0.41 0.036 0.14 0.31 0.0072 0.0024) *
        25) B5< 76.5 906  567 5 (0.24 0.012 0.13 0.052 0.16 0.37 0.012 0.011)  
          50) B3>=37.5 205   85 0 (0.59 0.015 0.059 0.02 0.11 0.18 0.02 0.015) *
          51) B3< 37.5 701  399 5 (0.14 0.011 0.16 0.061 0.18 0.43 0.01 0.01) *
      13) B4< 29.5 1282  843 3 (0.023 0.0016 0.094 0.26 0.26 0.34 0.0023 0.018) *
    7) B4< 17.5 754  202 3 (0.0013 0 0.017 0.73 0.06 0.14 0 0.049) *"""
])

def classify_single_zone(year, zone_number, area):
    # Prepare NDVI image as before
    lists = ee.ImageCollection("MODIS/061/MYD13Q1") \
        .select('NDVI') \
        .filterDate(f'{year}-01-01', f'{year}-12-31') \
        .sort('system:index', True) \
        .toList(23)
    def add_bands_reducer(a, b):
        return ee.Image(b).addBands(a)
    images = ee.Image(lists.slice(1).iterate(add_bands_reducer, lists.get(0)))
    input_bands = ['NDVI_4', 'NDVI_5', 'NDVI_3']
    classifier_bands = ['B4', 'B5', 'B3']
    images_for_classification = images.select(input_bands).rename(classifier_bands).multiply(0.0001).multiply(100)

    i = int(zone_number)
    zone = area.filter(ee.Filter.eq('region', i)).union()
    mask = ee.Image(0).byte().paint(zone, 1)
    tree = ee.String(treeString.get(i))
    classifier = ee.Classifier.decisionTree(tree)
    classified = images_for_classification.classify(classifier)
    result = classified.byte().updateMask(mask)
    return result

# --- implementation ---
year_2014 = 2014
baseline_2000 = 2000
label_2000 = f'AEZ_{baseline_2000}_zones'
zones_2000 = ee.FeatureCollection(os.path.join("users", username, label_2000))
cropped_2000 = classify_cropland_by_zone(cropland_mask_250m, zones_2000, [1, 2, 3, 4, 5]) #only cropped zones from cropland mask
area_2000 = cropped_2000
results_2014 = ee.ImageCollection([
    classify_single_zone(year_2014, i, area_2000) for i in range(1, 8) # 8 FAO AEZ
]).mosaic()


# Save results for later comparison
acma_results = {
    "2014": results_2014
}


We plot the result below. It takes time for the mapper to show the colored pixels given our resolution.

In [11]:

display(Markdown("""
| Value | Class Name (SC= single crop; DC= double crop) | Color  |
|       | season 1- Oct-Mar; season 2- May-Sep          |        |
|-------|-----------------------------------------------|--------|
| 0     | Irrigated, SC, season 2                       | Green  |
| 1     | Irrigated, SC, season 1                       | Yellow |
| 2     | Irrigated, DC,                                | Blue   |
| 3     | Irrigated, Continuous                         | Pink   |
| 4     | Rainfed, SC, season 2                         | Brown  |
| 5     | Rainfed, SC, season 1                         | Red    |
| 6     | Rainfed, DC,                                  | Orange |
| 7     | Rainfed, Continuous                           | Violet |
"""))

# Create map to visualize 2014 results
Map = geemap.Map(center=[0, 20], zoom=3)

# Add the 2014 classification results
vis_params = {
    'min': 0, 
    'max': 7, 
    'palette': ['green', 'yellow', 'blue', 'pink', 'brown', 'red', 'orange', 'violet']
}

Map.addLayer(results_2014, vis_params, 'ACMA 2014 Classification')

# Add zone boundaries for reference
Map.addLayer(zones_2000.style(color='white', fillColor='00000000', width=1), {}, 'AEZ Zones')

# Display the map
Map



| Value | Class Name (SC= single crop; DC= double crop) | Color  |
|       | season 1- Oct-Mar; season 2- May-Sep          |        |
|-------|-----------------------------------------------|--------|
| 0     | Irrigated, SC, season 2                       | Green  |
| 1     | Irrigated, SC, season 1                       | Yellow |
| 2     | Irrigated, DC,                                | Blue   |
| 3     | Irrigated, Continuous                         | Pink   |
| 4     | Rainfed, SC, season 2                         | Brown  |
| 5     | Rainfed, SC, season 1                         | Red    |
| 6     | Rainfed, DC,                                  | Orange |
| 7     | Rainfed, Continuous                           | Violet |


Map(center=[0, 20], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(child…

The ACMA code sample was made 10 years ago, and thus a lot of it is not working now. I had to modify some aspects to it so that it runs given the newer ee module versions.

Original code published by the authors is [here](https://code.earthengine.google.com/ce07c366433e44f4bc76def1b9efdbf3).

For context, I used the WEKA-trained decision tree that Xiong et al., 2017 developed. [WEKA](https://doi.org/10.1007/s12040-013-0339-2).

There are some points of uncertainty regarding the ACMA sample code that I was using and whether I interpreted them correctly:
1. Decision trees were all the same for each zone. Was this as intended?

2. Band references in the decision tree:
The decision tree references bands b3, b4, and b5. It's unclear whether these refer to original MODIS bands or are aliases created after preprocessing. Based on Figure 6 of the paper, I assumed these might correspond to NDVI temporal composites-- meaning b2 meant 2nd NDVI band of the 23, as they were used for decision thresholds in Figure 6 of the paper. Is this correct?

3. Decision tree class assignments:
The original decision trees appeared to consistently assign class 0 at most nodes, even when class probabilities indicated otherwise. This led to overly uniform classification results. I adjusted the tree to reflect the most probable class per node, which improved the classification outputs. If there's a different intended approach here, I’d be grateful for your insights.

4. Input FeatureCollection structure:
The original training FeatureCollection appears to be no longer publicly accessible. Based on your paper, I inferred that the dataset likely included a "region" attribute corresponding to FAO GAEZ zones 1–8, and that non-cropland areas were masked out — I’ve implemented both in my adaptation. However, the reference 250 m cropland mask mentioned seems unavailable, so I relied on the 2010 data as a proxy. Could you provide any guidance or clarification on this input structure?

And if I understand correctly, these sample codes in github/gee were intended for applying the trained trees to other years. Hence, to update ACMA to correctly map 2015 beyond, I would need this + update the baseline crop layer, which is from the 10 datasets mentioned in Table 1 from the paper + the new ones for 2017 onwards?

Hoping for clarification and your insights. I am really interested in building on your work, but I aim to do it as accurately as possible. Thank you for your time and consideration.


References:

FAO and IIASA. Global Agro Ecological Zones version 4 (GAEZ v4). [June 11, 2025]Accessed DATE. URL:  http://www.fao.org/gaez/

P. Thenkabail, P. Teluguntla, J Xiong, A. Oliphant, R. Massey (2016). NASA MEaSUREs Global
Food Security Support Analysis Data (GFSAD) Crop Mask 2010 Global 1 km V001. NASA EOSDIS
Land Processes DAAC. Retrieved from
https://doi.org/10.5067/MEaSUREs/GFSAD/GFSAD1KCM.001

Xiong, J., Thenkabail, P., Gumma, M., Teluguntla, P., Poehnelt, J., Congalton, R., Yadav, K., & Thau, D.
(2017). Automated cropland mapping of continental Africa using Google Earth Engine cloud computing
(Download Full text from: Http://dx.doi.org/10.1016/j.isprsjprs.2017.01.019). ISPRS Journal of
Photogrammetry and Remote Sensing, 126, 225–244.

